In [1]:
from src.inference.groq import ChatGroq
from tools import terminal_tool,save_tool,weather_tool,search_tool
from src.message import HumanMessage,SystemMessage
from src.agent.tool_generator import ToolGeneratorAgent
from os import environ
from dotenv import load_dotenv

load_dotenv()
api_key=environ.get('GROQ_API_KEY')
llm=ChatGroq('llama-3.1-70b-versatile',api_key,temperature=0)

In [2]:
from asyncio import create_task,wait,gather

In [3]:
async def tool_generator(query):
    agent=ToolGeneratorAgent(llm=llm,json=True)
    return agent.invoke(query)

In [4]:
task1=create_task(tool_generator('Create a tool that can find the weather data of a given location. Name it Weather Tool'))
task2=create_task(tool_generator('Create a tool that can give the latest news updates. Name it News Tool'))
results=await gather(task1,task2)

In [5]:
for result in results:
    print(result['tool'])

class Weather(BaseModel):
    location:str=Field(...,description="The location to get the weather data for.")

@tool("Weather Tool",args_schema=Weather)
def weather_tool(location:str):
    """
    Retrieves the current weather data for the given location using OpenWeatherMap API.
    """
    import requests
    import os
    import json

    api_key=os.environ.get('OPENWEATHERMAP_API_KEY')
    try:
        base_url="http://api.openweathermap.org/data/2.5/weather"
        params={
            'q':location,
            'appid':api_key,
            'units':'metric'
        }
        response=requests.get(base_url,params=params)
        response.raise_for_status()
        weather_data=response.json()
        return json.dumps(weather_data,indent=4)
    except requests.exceptions.HTTPError as errh:
        return f"HTTP Error: {errh}"
    except requests.exceptions.ConnectionError as errc:
        return f"Error Connecting: {errc}"
    except requests.exceptions.Timeout as errt:
        ret